In [35]:
from pathlib import Path
import numpy as np


def read_input(filename):
    with Path(filename).open() as f:
        lines = f.readlines()
        bingo_numbers = [int(num) for num in lines[0].split(',')]
        board_lines = "".join(lines[2:])
        boards_info = board_lines.split("\n\n")
        boards = np.array([np.array([[int(num) for num in board_line.split(" ") if num] for board_line in board_info.strip("\n").split("\n")]) for board_info in boards_info if board_info])
        return bingo_numbers, boards 


In [36]:
bingo_numbers, boards = read_input("days/day-04/input.txt")

In [39]:
bingo_record = np.zeros(boards.shape)

In [59]:
bingo_record = np.zeros(boards.shape)
for number in bingo_numbers:
    bingo_record += (boards == number).astype(int)
    if np.any(bingo_record.sum(axis=1) == 5) or np.any(bingo_record.sum(axis=2) == 5):
        print("Bingo! Bingo! Bingo!")
        winning_board_index = [np.any(record.sum(axis=0) == 5) or np.any(record.sum(axis=1) == 5) for record in bingo_record].index(True)
        winning_board = boards[winning_board_index]
        winning_board_record = bingo_record[winning_board_index]
        unmarked_numbers = winning_board[winning_board_record == 0]
        solution = np.sum(unmarked_numbers) * number
        print(solution)
        break

Bingo! Bingo! Bingo!
22680


In [65]:
bingo_record = np.zeros(boards.shape)
for number in bingo_numbers:
    bingo_record += (boards == number).astype(int)

    losing_boards = [not(np.any(record.sum(axis=0) == 5) or np.any(record.sum(axis=1) == 5)) for record in bingo_record]
    if sum(losing_boards) == 1:
        losing_board_index = losing_boards.index(True)
        losing_board = boards[losing_board_index]

    if sum(losing_boards) == 0:
        losing_board_record = bingo_record[losing_board_index]
        unmarked_numbers = losing_board[losing_board_record == 0]
        solution = np.sum(unmarked_numbers) * number
        print(solution)
        break

16168


In [128]:
from pathlib import Path
import numpy as np

def parse_board(board):
    return [[int(num) for num in line.split()] for line in board.split("\n")]


def read_input(filename):
    with Path(filename).open() as f:
        lines = f.readlines()
        bingo_numbers = list(map(int, lines[0].split(",")))
        boards = map(parse_board, "".join(lines[1:]).strip("\n").split("\n\n"))
        return bingo_numbers, np.array(list(boards))

def find_bingo_winners(bingo_record):
    wins_by_rows = np.any(np.all(bingo_record, axis=1), axis=1)
    wins_by_cols = np.any(np.all(bingo_record, axis=2), axis=1)
    return wins_by_rows | wins_by_cols

def find_first_winner(bingo_numbers, bingo_boards):
    bingo_record = np.zeros_like(bingo_boards, dtype=bool)
    for number in bingo_numbers:
        bingo_record += (bingo_boards == number)
        if any(winners := find_bingo_winners(bingo_record)):
            return number, bingo_boards[winners][0], bingo_record[winners][0]

def find_last_winner(bingo_numbers, bingo_boards):
    bingo_record = np.zeros_like(bingo_boards, dtype=bool)
    for number in bingo_numbers:
        bingo_record += (bingo_boards == number)
        if all(winners := find_bingo_winners(bingo_record)):
            return number, bingo_boards[last_round_losers][0], bingo_record[last_round_losers][0]
        last_round_losers = ~winners

def compute_solution(number, board, board_record):
    return np.sum(board[board_record == 0]) * number

In [129]:
bingo_numbers, bingo_boards = read_input("days/day-04/input.txt")
print(compute_solution(*find_first_winner(bingo_numbers, bingo_boards)))
print(compute_solution(*find_last_winner(bingo_numbers, bingo_boards)))

22680
16168
